#Install

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 41.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 11.8 MB/s eta 0:00:00


In [ ]:
import importlib
import transformers

# Textual inference

In [ ]:
from nltk.metrics import ConfusionMatrix, scores

In [ ]:
# SemDL package includes utility functions
!rm -fr SemDL # helps to rerun this cell witthout errors, if recloning needed
! git clone https://github.com/kovvalsky/SemDL.git

Cloning into 'SemDL'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 29 (delta 4), reused 9 (delta 0), pack-reused 0
Unpacking objects: 100% (29/29), 62.58 KiB | 942.00 KiB/s, done.


In [ ]:
import SemDL.reasoning
importlib.reload(SemDL.reasoning) # useful when updating the module files
from SemDL.reasoning import gen_syllogism, load_tok_model, predict_nli

In [ ]:
# generating a couple of syllogisms
for name_label, (p1, p2, c) in gen_syllogism('logicians', 'linguists', 'engineers', figures="1", annotated=True):
    if "neutral" not in name_label:
        print(f"{name_label}\n{p1}\n{p2}\n{'':->30}\n{c}\n")

f1-aaa-entailment
All logicians are engineers
All linguists are logicians
------------------------------
All linguists are engineers

f1-eia-contradiction
No logicians are engineers
Some linguists are logicians
------------------------------
All linguists are engineers

f1-aie-contradiction
All logicians are engineers
Some linguists are logicians
------------------------------
No linguists are engineers

f1-eae-entailment
No logicians are engineers
All linguists are logicians
------------------------------
No linguists are engineers

f1-aii-entailment
All logicians are engineers
Some linguists are logicians
------------------------------
Some linguists are engineers

f1-eai-contradiction
No logicians are engineers
All linguists are logicians
------------------------------
Some linguists are engineers

f1-aao-contradiction
All logicians are engineers
All linguists are logicians
------------------------------
Some linguists are not engineers

f1-eio-entailment
No logicians are engineers


In [ ]:
model_name = 'ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli'
tokenizer, model_cpu = load_tok_model(model_name)

Some weights of the model checkpoint at ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
predict_nli(tokenizer, model_cpu, ("All logicians are engineers. All linguists are logicians.", "All linguists are engineers"))

{'label_index': 2,
 'label': 'contradiction',
 'prob': 0.8995290994644165,
 'probs': {'entailment': 0.06602967530488968,
  'neutral': 0.03444121032953262,
  'contradiction': 0.8995290994644165}}

In [ ]:
pred_ref = dict()
for name_label, (p1, p2, c) in gen_syllogism('logicians', 'linguists', 'engineers', annotated=True):
    name, ref_label = name_label.rsplit('-', 1)
    pred = predict_nli(tokenizer, model_cpu, (f"{p1}. {p2}.", f"{c}."))
    pred_ref[name] = pred['label'], ref_label

In [ ]:
# Evaluation
preds, refs = zip(*pred_ref.values())
print(ConfusionMatrix(refs, preds))

acc = scores.accuracy(refs, preds)
print(f"Accuracy = {acc}")

              |   c         |
              |   o         |
              |   n         |
              |   t   e     |
              |   r   n     |
              |   a   t     |
              |   d   a   n |
              |   i   i   e |
              |   c   l   u |
              |   t   m   t |
              |   i   e   r |
              |   o   n   a |
              |   n   t   l |
--------------+-------------+
contradiction | <10>  .   5 |
   entailment |   5 <10>  . |
      neutral | 140  52 <34>|
--------------+-------------+
(row = reference; col = test)

Accuracy = 0.2109375


In [ ]:
# Analysis
for name, (p1, p2, c) in gen_syllogism('logicians', 'linguists', 'engineers'):
    pred, ref = pred_ref[name]
    if pred != ref and ref in "entailment contradiction":
        print(f"{name}\t{ref.upper()}\t{pred}\n{p1}\n{p2}\n{'':->30}\n{c}\n")

f1-aaa	ENTAILMENT	contradiction
All logicians are engineers
All linguists are logicians
------------------------------
All linguists are engineers

f1-eae	ENTAILMENT	contradiction
No logicians are engineers
All linguists are logicians
------------------------------
No linguists are engineers

f1-aii	ENTAILMENT	contradiction
All logicians are engineers
Some linguists are logicians
------------------------------
Some linguists are engineers

f1-eai	CONTRADICTION	neutral
No logicians are engineers
All linguists are logicians
------------------------------
Some linguists are engineers

f1-aao	CONTRADICTION	neutral
All logicians are engineers
All linguists are logicians
------------------------------
Some linguists are not engineers

f2-aee	ENTAILMENT	contradiction
All engineers are logicians
No linguists are logicians
------------------------------
No linguists are engineers

f2-eae	ENTAILMENT	contradiction
No engineers are logicians
All linguists are logicians
----------------------------